In [1]:
import numpy as np
import pandas as pd
import random as rd
import re
import matplotlib.pyplot as plt
import seaborn as sns

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.decomposition import TruncatedSVD
from CLARA import CLARA
from sklearn.metrics import silhouette_score
%matplotlib inline

path = '/Users/Rizal Maulana/Downloads/semester 7/Skripsi/datas/'
stoplist = list(stopwords.words('english'))
lemma = WordNetLemmatizer()

# Read Document

## Document Original

In [11]:
data = pd.read_csv(path+'articles1.csv')
mask = (data['date'] > '2017-01-01') & (data['date'] <= '2017-01-03')
# 107, 226, 377

korpus = data[mask]['content']
korpus.index = range(len(korpus))
del data, mask
len(korpus)

226

## Document Summary

In [105]:
data = pd.read_csv(path+'text_summarizer_data.csv',index_col=0)
korpus = data.loc[:106,'content']
len(korpus)

107

## Document Classification

In [124]:
data = pd.read_csv(path+'dataset.csv')
index_ = rd.sample([i for i in data.index],107)
korpus = data.loc[index_,'text']
class_ = data.loc[index_,'label']
korpus.index,class_ = range(len(korpus)), range(len(class_))
len(korpus)

107

# Text Prepocessing

In [12]:
doc_term = []
for doc in korpus:
    token = re.findall(r'\b[A-Za-z]{2,}',doc)
    term = []
    for word in token:
        if word.lower() not in stoplist:
            term.append(lemma.lemmatize(word))
    doc_term.append(term)
    
doc_term[0:5]

[['SEOUL',
  'South',
  'Korea',
  'North',
  'Korea',
  'leader',
  'Kim',
  'said',
  'Sunday',
  'country',
  'making',
  'final',
  'preparation',
  'conduct',
  'first',
  'test',
  'intercontinental',
  'ballistic',
  'missile',
  'bold',
  'statement',
  'le',
  'month',
  'inauguration',
  'Donald',
  'Trump',
  'Although',
  'North',
  'Korea',
  'conducted',
  'five',
  'nuclear',
  'test',
  'last',
  'decade',
  'ballistic',
  'missile',
  'test',
  'alone',
  'although',
  'habitually',
  'threatens',
  'attack',
  'United',
  'States',
  'nuclear',
  'weapon',
  'country',
  'never',
  'intercontinental',
  'ballistic',
  'missile',
  'ICBM',
  'annual',
  'New',
  'Year',
  'Day',
  'speech',
  'broadcast',
  'North',
  'KCTV',
  'Sunday',
  'Mr',
  'Kim',
  'spoke',
  'proudly',
  'stride',
  'said',
  'country',
  'made',
  'nuclear',
  'weapon',
  'ballistic',
  'missile',
  'program',
  'said',
  'North',
  'Korea',
  'would',
  'continue',
  'bolster',
  'weapon',
 

# Pembobotan Term

In [13]:
term = np.unique(sum(doc_term,[]))
columns = ['doc'+str(i) for i in range(len(doc_term))]
dtm = pd.DataFrame(0,index=term,columns=columns)

for i,doc in enumerate(doc_term):
    for term in doc:
        dtm.loc[term,'doc'+str(i)] += 1
    
dtm['frequency'] = (dtm > 0).values.sum(axis=1)
dtm.head()

,doc0,doc1,doc2,doc3,doc4,doc5,doc6,doc7,doc8,doc9,...,doc217,doc218,doc219,doc220,doc221,doc222,doc223,doc224,doc225,frequency
ABC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6
ACA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
ACC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
AE,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
AFP,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


# TF-IDF

In [14]:
def tfidf(dtm):
    frequency = dtm['frequency']
    dtm = dtm.drop(columns='frequency')
    np.seterr(divide = 'ignore') 
    
    tf = pd.DataFrame(np.where(dtm.values > 0, np.log10(dtm) + 1,0))
    idf = pd.DataFrame(np.log10(len(dtm.columns)/frequency))
    wtd = pd.DataFrame(tf.values*idf.values)
    jumlah = np.sqrt(np.sum(np.square(wtd)))
    normalization = wtd.div(jumlah,axis=1)
    normalization.columns, normalization.index = dtm.columns, dtm.index
    
    return normalization

dtm_tfidf = tfidf(dtm)
dtm_tfidf = dtm_tfidf.transpose()
dtm_tfidf.head()

,ABC,ACA,ACC,AE,AFP,AFT,AG,AIDS,AME,AMERICA,...,young,younger,youngest,youth,youthful,zany,zealot,zero,zilliopzazillion,zone
doc0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
doc1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
doc2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
doc3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
doc4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Feature Transformation

In [15]:
svd = TruncatedSVD(n_components=200).fit(dtm_tfidf) # 95:0.9205, 200:0.9273, 330:0.9202
print('Persentasi SVD : {}'.format(svd.explained_variance_ratio_.sum()))
svd = svd.transform(dtm_tfidf)

columns = ['SVD_'+str(i) for i in range(len(svd[0]))]
svd = pd.DataFrame(svd, index=range(len(dtm_tfidf.index)), columns=columns)
svd.head()

Persentasi SVD : 0.9268536905699742


,SVD_0,SVD_1,SVD_2,SVD_3,SVD_4,SVD_5,SVD_6,SVD_7,SVD_8,SVD_9,...,SVD_190,SVD_191,SVD_192,SVD_193,SVD_194,SVD_195,SVD_196,SVD_197,SVD_198,SVD_199
0,0.206929,-0.017439,0.026747,0.005107,0.036735,-0.034263,-0.037286,-0.129929,-0.019142,-0.015636,...,-0.083544,0.029544,-0.030561,-0.029755,-0.040759,-0.009378,0.125257,-0.010413,0.023829,-0.114802
1,0.069422,-0.054584,0.003269,-0.004005,-0.004938,-0.014505,-0.032864,0.014554,0.014110,0.003325,...,0.045607,-0.074809,-0.038096,-0.005409,0.058056,0.008570,-0.006554,-0.022281,0.045117,0.002233
2,0.188073,-0.012358,0.027098,-0.000414,0.039645,-0.042826,-0.030995,-0.130523,0.004857,-0.027679,...,0.043664,-0.030189,0.043286,0.094393,0.038946,0.004818,0.071687,-0.111346,0.082297,0.013772
3,0.274471,0.062676,-0.021441,0.078273,0.013133,-0.071422,-0.028070,-0.108380,0.067916,-0.063299,...,0.017917,-0.089090,-0.113916,-0.003453,0.039638,0.098847,0.069700,0.009643,-0.023464,-0.014863
4,0.221781,-0.025162,0.004117,0.011636,0.014806,-0.052046,-0.043214,0.034497,0.041188,0.006300,...,0.032931,-0.003645,-0.053987,-0.073251,0.068540,0.041331,-0.028657,0.005812,-0.009017,0.072037


# Optimal CLARA

In [8]:
def optimal(df,awal,akhir):
    eval_,cluster,labels = [],[],{}
    
    for i in range(awal,akhir):
        clara = CLARA(n_clusters=i)
        label = clara.fit_transform(df)
        sil = np.round(silhouette_score(df, label),3)
        eval_.append(sil)
        cluster.append(i)
        labels[i] = label
#         print('{} Cluster, Silhouette Score {}'.format(i,sil))
        print(sil)
        
    return eval_,cluster,labels

# Silhouette Score

In [25]:
ss,cluster,labels = optimal(svd,2,3)
# plt.figure(figsize=(10,5))
# sns.pointplot(x=cluster, y=ss)
# plt.title('Silhouette Score with {} data'.format(len(svd)))
# plt.tight_layout()
# plt.xlabel('Cluster')
# plt.ylabel('Silhouette Score')
# plt.show()
dtm_c = pd.Series(labels[2],index=dtm_tfidf.index)
np.unique(dtm_c,return_counts=True)

-0.003


(array([0, 1], dtype=int64), array([  1, 225], dtype=int64))

In [17]:
dtm_c = pd.Series(labels[2],index=dtm_tfidf.index)
np.unique(dtm_c,return_counts=True)

(array([0, 1], dtype=int64), array([  2, 224], dtype=int64))

In [26]:
dtm_c.loc[dtm_c == 0]

doc14    0
dtype: int64

# Penghapusan Dokumen

In [28]:
df_test = svd.drop([svd.index[14]])
df_test.index = range(len(df_test))
ss,cluster,lab = optimal(df_test,2,3)
dtm_c = pd.Series(lab[2],index=df_test.index)
np.unique(dtm_c,return_counts=True)

0.011


(array([0, 1], dtype=int64), array([224,   1], dtype=int64))

# Topic Extraction

## Pembobotan pada Cluster

In [9]:
dtm_te = pd.DataFrame(0,index=dtm.index, columns=np.unique(dtm_c))
for i in np.unique(dtm_c):
    group = dtm_c.groupby(dtm_c)
    dtm_te[i] = (dtm[group.get_group(i).index] > 0).values.sum(axis=1)
    
dtm_te.columns = ['Cluster '+str(i) for i in dtm_te.columns]
dtm_te.head()

,Cluster 0,Cluster 1
ABC,0,6
ACA,0,1
ACC,1,1
AE,0,1
AFP,0,2


## TF-ICF

In [10]:
frequency = pd.Series((dtm_te > 0).values.sum(axis=1),index=dtm_te.index)
icf = pd.DataFrame(np.log10(len(dtm_te.columns)/frequency))
wtd = dtm_te.values*icf.values
dtm_tficf = pd.DataFrame(wtd, index=dtm_te.index, columns=dtm_te.columns)
dtm_tficf.head()

,Cluster 0,Cluster 1
ABC,0.0,1.80618
ACA,0.0,0.30103
ACC,0.0,0.00000
AE,0.0,0.30103
AFP,0.0,0.60206


## Overall Topic Accuracy

In [11]:
def overall(dtm_tficf,atas,bawah):
    for l in range(atas,bawah):
        topic_extraction,term_acc = [],[]
        for i in dtm_tficf.columns:
            topic_extraction.append(dtm_tficf.nlargest(l,i))
            
        for k,i in enumerate(topic_extraction):
            term_acc.append(i['Cluster '+str(k)]/i.sum(axis=1))

        overall = round((sum([i.sum()/l for i in term_acc]))/len(dtm_te.columns),3)
        print('Jumlah Topik {} : {}'.format(l,overall))
        
overall(dtm_tficf,1,11)

Jumlah Topik 1 : 1.0
Jumlah Topik 2 : 1.0
Jumlah Topik 3 : 1.0
Jumlah Topik 4 : 1.0
Jumlah Topik 5 : 1.0
Jumlah Topik 6 : 1.0
Jumlah Topik 7 : 1.0
Jumlah Topik 8 : 1.0
Jumlah Topik 9 : 1.0
Jumlah Topik 10 : 1.0


In [12]:
for k,i in enumerate(dtm_tficf.columns):
    print('Topic in Cluster {}:\n{}'.format(k,list(dtm_tficf.nlargest(10,i).index)))

Topic in Cluster 0:
['Oct', 'solidarity', 'Baldwin', 'Coalition', 'Ten', 'Walmart', 'Wikileaks', 'Writers', 'advised', 'analyzed']
Topic in Cluster 1:
['final', 'Bush', 'economy', 'Richard', 'due', 'European', 'elected', 'fan', 'presidency', 'crisis']


## Pengujian Truncated SVD

In [8]:
uji = [50,60,70,80,90,100]
for j in uji:
    svd = TruncatedSVD(n_components=j, n_iter=50).fit(dtm_tfidf) # 77:0.9111, 155:0.9110, 260:0.9119, 340:0.9122
    print('Persentasi SVD : {}'.format(svd.explained_variance_ratio_.sum()))
    svd = svd.transform(dtm_tfidf)

    columns = ['SVD_'+str(i) for i in range(len(svd[0]))]
    svd = pd.DataFrame(svd, index=range(len(dtm_tfidf.index)), columns=columns)
    print(j)
    for k in range(0,10):
        clara = CLARA(n_clusters=3,max_iter=100)
        label = clara.fit_transform(svd)
        sil = np.round(silhouette_score(svd, label),5)
        print(k,sil)

Persentasi SVD : 0.7302137017250061
50
0 0.1518
1 -0.46291
2 -0.42852
3 -0.34663
4 -0.46768
5 -0.4619
6 -0.42363
7 0.20243
8 -0.00475
9 -0.2336
10 -0.42246
Persentasi SVD : 0.8065282106399536
60
0 -0.33454
1 -0.44181
2 -0.40577
3 0.05191
4 0.11583
5 -0.37458
6 -0.43493
7 0.1318
8 -0.37427
9 0.0841
10 0.09759
Persentasi SVD : 0.8713119029998779
70
0 -0.40993
1 0.06886
2 -0.37765
3 -0.16274
4 0.00799
5 -0.40118
6 0.05834
7 -0.2678
8 -0.39947
9 0.17767
10 0.17795
Persentasi SVD : 0.926230788230896
80
0 -0.37722
1 0.10022
2 0.02744
3 -0.37722
4 -0.37831
5 -0.35542
6 -0.34977
7 0.04253
8 -0.03419
9 -0.01484
10 -0.38158
Persentasi SVD : 0.9665976762771606
90
0 -0.03287
1 -0.02875
2 0.00314
3 -0.17985
4 -0.2397
5 -0.35214
6 -0.34229
7 -0.35518
8 -0.35163
9 -0.05149
10 0.01661
Persentasi SVD : 0.9909952282905579
100
0 -0.1432
1 -0.14568
2 -0.20807
3 0.03777
4 0.01574
5 -0.20894
6 -0.32988
7 -0.13584
8 -0.07944
9 -0.32994
10 -0.13248
